<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import math

In [2]:
# Importing the excel
!unzip punch_short.xlsx
dataset = pd.read_excel('punch_short.xlsx',parse_dates=['Date'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d'))
dataset2=pd.read_excel('Master Data.xlsx')
dataset3=pd.read_excel('Shift_Timings.xlsx')

Archive:  punch_short.xlsx
replace [Content_Types].xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: [Content_Types].xml     
  inflating: _rels/.rels             
  inflating: xl/workbook.xml         
  inflating: xl/_rels/workbook.xml.rels  
  inflating: xl/worksheets/sheet1.xml  
  inflating: xl/theme/theme1.xml     
  inflating: xl/styles.xml           
  inflating: xl/sharedStrings.xml    
  inflating: xl/externalLinks/externalLink1.xml  
  inflating: xl/calcChain.xml        
  inflating: docProps/core.xml       
  inflating: docProps/app.xml        
  inflating: xl/externalLinks/_rels/externalLink1.xml.rels  


In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [5]:
dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')
# dataset3['date'] = pd.to_datetime(dataset3['date'], errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
a1=str(dataset.iloc[0]['Date'])
a2=str(dataset.iloc[-1]['Date'])
d1 = datetime.strptime(a1[0:10], "%Y-%m-%d")
d2 = datetime.strptime(a2[0:10], "%Y-%m-%d")
date_diff = abs((d2 - d1).days)+1  

In [7]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l_shift=[]
for i in range(len(dataset3)):
  x_=str(dataset3.iloc[i]['Start'])
  y_=str(dataset3.iloc[i]['End'])

  if len(x_)<=5:
    x_=x_.zfill(6)
  if len(y_)<=5:
    y_=y_.zfill(6)

  x_ = datetime.strptime(x_, "%H%M%S")
  x_.strftime("%I:%M %p")
  y_ = datetime.strptime(y_, "%H%M%S")
  y_.strftime("%I:%M %p")

  presentday = pd.to_datetime('20221001', errors='coerce')
  tomorrow = pd.to_datetime('20221002', errors='coerce')
  if dataset3.iloc[i]['End date']=='same':
    l_shift.append([datetime.combine(presentday, y_.time()) - datetime.combine(presentday, x_.time())])
  else:
    l_shift.append([datetime.combine(tomorrow, y_.time()) - datetime.combine(presentday, x_.time())])    

In [8]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  x=str(dataset.iloc[i]['Time'])
  y=str(dataset.iloc[j]['Time'])

  if len(x)<=5:
    x=x.zfill(6)
  if len(y)<=5:
    y=y.zfill(6)

  x = datetime.strptime(x, "%H%M%S")
  x.strftime("%I:%M %p")
  y = datetime.strptime(y, "%H%M%S")
  y.strftime("%I:%M %p")

  l.append([dataset.iloc[i]['E.Code'], dataset.iloc[i]['Date'].date(), dataset.iloc[i]['IN_OUT'], int(dataset.iloc[i]['Time']), datetime.combine(dataset.iloc[j]['Date'], y.time()) - datetime.combine(dataset.iloc[i]['Date'], x.time())])  
  i=i+2

In [9]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][4] = l[i][4].__str__().replace(":",".")
  s=l[i][4]
  l[i][4]=s[0:-3]

for i in range(len(l_shift)):
  l_shift[i][0] = l_shift[i][0].__str__().replace(":",".")
  s=l_shift[i][0]
  l_shift[i][0]=float(s[0:-3])

In [10]:
data3= pd.DataFrame(l_shift,columns=['Difference'])
dataset3= pd.concat([dataset3,data3],axis=1)

In [11]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'P20' not in str(l[i][2]):
    l2.append(l[i]) 

In [12]:
for i in range(len(l2)):
  if l2[i][0]==5656:
    print(l2[i])

[5656, datetime.date(2022, 10, 3), 'P10', 83200, '1 day, 9.30']
[5656, datetime.date(2022, 10, 6), 'P10', 83600, '9.16']
[5656, datetime.date(2022, 10, 7), 'P10', 83200, '9.41']
[5656, datetime.date(2022, 10, 10), 'P10', 83400, '10.02']
[5656, datetime.date(2022, 10, 11), 'P10', 83900, '9.23']
[5656, datetime.date(2022, 10, 12), 'P10', 83800, '9.53']
[5656, datetime.date(2022, 10, 13), 'P10', 84000, '9.20']
[5656, datetime.date(2022, 10, 14), 'P10', 85600, '9.12']


In [13]:
for i in range(len(l2)):
  if 'd' in l2[i][4]:
    s=l2[i][4]
    s=float(s[8:len(s)])
    l2[i][4]=24+s

In [14]:
#Converting string type to float for Time column
for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:                      
    print(l2[i][0],l2[i][4])

for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:                      
    print(l2[i][0],l2[i][4])

In [17]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):  
  frac, whole = math.modf(l2[i][4])
  mins=int(whole*60+frac*100)


  
  def binarySearch(arr, l, r, x):

    while l <= r:

      mid = l + (r - l) // 2

      # Check if x is present at mid
      if arr[mid] == x:
        return mid

      # If x is greater, ignore left half
      elif arr[mid] < x:
        l = mid + 1

      # If x is smaller, ignore right half
      else:
        r = mid - 1

    # If we reach here, then the element
    # was not present
    return -1


  # Driver Code
  arr = [int(i) for i in dataset2['E Code']]
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, int(l2[i][0]))

  if result != -1:
      l_dest=dataset2.iloc[result]['Location']

  


  #1F A-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[0]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[0]['Start']-3000) and '1F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[0]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #1F B-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[1]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[1]['Start']-3000) and '1F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[1]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #1F C-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[2]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[2]['Start']-3000) and '1F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[2]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #HO G-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[15]['Start']+10000)>=l2[i][3]>=(dataset3.iloc[15]['Start']-10000) and 'Head' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[15]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #2F A-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[3]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[3]['Start']-3000) and '2F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[3]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #2F B4-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[4]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[4]['Start']-3000) and '2F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[4]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #2F C-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[5]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[5]['Start']-3000) and '2F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[5]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #2F G-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[6]['Start']+10000)>=l2[i][3]>=(dataset3.iloc[6]['Start']-10000) and '2F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[6]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #3F A-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[7]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[7]['Start']-3000) and '3F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[7]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #3F B4-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[8]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[8]['Start']-3000) and '3F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[8]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #3F C-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[9]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[9]['Start']-3000) and '3F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[9]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #3F G-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[10]['Start']+10000)>=l2[i][3]>=(dataset3.iloc[10]['Start']-10000) and '3F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[10]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #4F A-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[11]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[11]['Start']-3000) and '4F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[11]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #4F B-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[12]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[12]['Start']-3000) and '4F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[12]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #4F C-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[13]['Start']+1500)>=l2[i][3]>=(dataset3.iloc[13]['Start']-3000) and '4F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[13]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  #4F G-Shift
  if l2[i][2]=='P10' and (dataset3.iloc[14]['Start']+10000)>=l2[i][3]>=(dataset3.iloc[14]['Start']-10000) and '4F' in l_dest:
    frac1, whole1 = math.modf(dataset3.iloc[14]['Difference'])
    mins_dif=int(whole1*60+frac1*100)
    mins_dif+=30
    if mins>mins_dif:
      mins=mins-mins_dif
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f a
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f a
1f a
1f b
1f b
1f a
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f c
1f b
1f c
1f a
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f c
1f b
1f c
1f c
1f b
1f c
1f b
1f c
1f b
1f b
1f b
1f c
1f b
1f b
1f a
1f b
1f b
1f c
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f a
1f b
1f b
1f b
1f b
1f c
1f c
1f b
1f c
1f c
1f c
1f b
1f b
1f c
1f b
1f b
1f b
1f c
1f c
1f a
1f c
1f c
1f b
1f c
1f b
1f b
1f a
1f b
1f c
1f b
1f b
1f b
1f a
1f b
1f b
1f b
1f b
1f c
1f b
1f b
1f b
1f b
1f b
1f b
1f a
1f b
1f b
1f b
1f b
1f c
1f b
1f c
1f b
1f b
1f b
1f c
1f b
1f c
1f b
1f b
1f c
1f c
1f c
1f c
1f b
1f c
1f b
1f b


In [19]:
#Removing redundant rows and storing in new data-structure
l4=[]
for i in range(len(l3)):
  if l3[i][5]>0:
    l4.append(l3[i]) 

In [20]:
for i in range(len(l4)):
  if len(l4[i])==8:
    print(l4[i][0],l4[i][1],l4[i][4],l4[i][5],l4[i][6],l4[i][7])

In [21]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l4, columns=['E.Code','Date','IN/OUT','Time','Total Time','OT'])

In [22]:
df_copy=df.copy(deep=True)

In [23]:
df

,E.Code,Date,IN/OUT,Time,Total Time,OT
0,637,2022-10-01,P10,144000,13.00,3.40
1,646,2022-10-01,P10,144200,12.59,3.39
2,816,2022-10-01,P10,144600,12.54,3.33
3,1363,2022-10-01,P10,143200,13.08,3.48
4,2100,2022-10-01,P10,143800,13.02,3.42
...,...,...,...,...,...,...
776,11828,2022-10-14,P10,75600,9.48,0.28
777,11836,2022-10-14,P10,83400,9.46,0.26
778,11837,2022-10-14,P10,84000,10.48,1.28
779,11000322,2022-10-14,P10,81200,10.11,0.51


In [24]:
#Droping an unnecesary column
df=df.drop(['IN/OUT','Time','Total Time'], axis=1)

In [25]:
ll1=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df)):
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, df.iloc[i]['E.Code'])

  if result != -1:
      ll1.append([dataset2.iloc[result]['Full Name'], dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
  else:
      ll1.append(['NaN','NaN','NaN','NaN'])


In [26]:
# Converting 2d list to dataframe to apply CRUD opertations
dff = pd.DataFrame(ll1, columns=['Full Name','Designation','Operation','Division','Department'])

In [27]:
dff

,Full Name,Designation,Operation,Division,Department
0,Dharamvir .,Staff,Manesar Plant,Production Body Manesar,Paint
1,C Senthil Kumar,Staff,Manesar Plant,Production Body Manesar,Assembly Frame
2,Shaikh Arshad Mahebubsab,Staff,Manesar Plant,Production Body Manesar,Assembly Frame
3,Ravi Kumar Adile,Staff,Manesar Plant,Production Body Manesar,Assembly Frame
4,Anand Kumar Moria,Staff,Manesar Plant,Production Body Manesar,Assembly Frame
...,...,...,...,...,...
776,Ashwani Kumar,Executive,Logistics Planning & Control,Logistics,HO Corporate Logistics
777,Harmanpreet Singh Bawa,Executive Trainee,Sales & Marketing,Sales Resource Quality & Training,Dealer Development
778,Rishabh Mittal,Executive Trainee,Customer Service,CS Technology & Customer Relations,Skill Enhancement
779,Akshat Malik,Fixed Term Associate,Sales & Marketing,Sales Resource Quality & Training,Dealer Development


In [28]:
df_final = pd.concat([df['E.Code'], dff, df[['Date','OT']]], axis=1)

In [29]:
#double shift
l_double_shift=[]
for i in range(len(df_copy)):
  if df_copy.iloc[i]['Time']>53000 and df_copy.iloc[i]['Time']<63000:
    if df_copy.iloc[i]['Total Time']>=17.50:
      l_double_shift.append([df_copy.iloc[i]['E.Code'],df_copy.iloc[i]['Date'],'A-B'])
  elif df_copy.iloc[i]['Time']>143000 and df_copy.iloc[i]['Time']<150000:
    if df_copy.iloc[i]['Total Time']>=15.50:
      l_double_shift.append([df_copy.iloc[i]['E.Code'],df_copy.iloc[i]['Date'],'B-C'])
  elif df_copy.iloc[i]['Time']>230000 and df_copy.iloc[i]['Time']<234000:
    if df_copy.iloc[i]['Total Time']>=15.50:
      l_double_shift.append([df_copy.iloc[i]['E.Code'],df_copy.iloc[i]['Date'],'C-A'])

In [30]:
l_double_shift=pd.DataFrame(l_double_shift,columns=['E.Code','Date','Double-Shift'])

In [31]:
l_double_shift2=[]
for i in range(len(l_double_shift)):
  for j in range(len(df_final)):
    if l_double_shift.iloc[i]['E.Code']==df_final.iloc[j]['E.Code'] and l_double_shift.iloc[i]['Date']==df_final.iloc[j]['Date']:
      l_double_shift2.append([df_final.iloc[j]['E.Code'],df_final.iloc[j]['Full Name'], df_final.iloc[j]['Designation'],df_final.iloc[j]['Operation'],df_final.iloc[j]['Division'],df_final.iloc[j]['Department'],df_final.iloc[j]['Date'],l_double_shift.iloc[i]['Double-Shift']])

In [32]:
l_double_shift2=pd.DataFrame(l_double_shift2,columns=['E.Code','Full Name','Designation','Operation','Division','Department','Date','Double-Shift'])

In [33]:
""" Printing dataframe (E.Code, Designation, Opr., Division, Dept., Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""
df_final

,E.Code,Full Name,Designation,Operation,Division,Department,Date,OT
0,637,Dharamvir .,Staff,Manesar Plant,Production Body Manesar,Paint,2022-10-01,3.40
1,646,C Senthil Kumar,Staff,Manesar Plant,Production Body Manesar,Assembly Frame,2022-10-01,3.39
2,816,Shaikh Arshad Mahebubsab,Staff,Manesar Plant,Production Body Manesar,Assembly Frame,2022-10-01,3.33
3,1363,Ravi Kumar Adile,Staff,Manesar Plant,Production Body Manesar,Assembly Frame,2022-10-01,3.48
4,2100,Anand Kumar Moria,Staff,Manesar Plant,Production Body Manesar,Assembly Frame,2022-10-01,3.42
...,...,...,...,...,...,...,...,...
776,11828,Ashwani Kumar,Executive,Logistics Planning & Control,Logistics,HO Corporate Logistics,2022-10-14,0.28
777,11836,Harmanpreet Singh Bawa,Executive Trainee,Sales & Marketing,Sales Resource Quality & Training,Dealer Development,2022-10-14,0.26
778,11837,Rishabh Mittal,Executive Trainee,Customer Service,CS Technology & Customer Relations,Skill Enhancement,2022-10-14,1.28
779,11000322,Akshat Malik,Fixed Term Associate,Sales & Marketing,Sales Resource Quality & Training,Dealer Development,2022-10-14,0.51


In [34]:
df_final_hr_min=df_final.copy(deep=True)
l_time1=[]
for i in df_final_hr_min.index:
      minutes, hours = math.modf(df_final_hr_min['OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time1.append(ot_time)
df_final_hr_min.drop(['OT'],axis=1)
df_final_hr_min['OT']=l_time1

In [38]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code'],as_index=False).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code'],as_index=False).count()
groupby_operation_sum = df_final.groupby(['Operation'],as_index=False).sum()
groupby_operation = df_final.groupby(['Operation'],as_index=False).count()
groupby_division_sum = df_final.groupby(['Division'],as_index=False).sum()
groupby_division = df_final.groupby(['Division'],as_index=False).count()
groupby_designation_sum = df_final.groupby(['Designation'],as_index=False).sum()
groupby_designation = df_final.groupby(['Designation'],as_index=False).count()


In [42]:
Operation_count = pd.DataFrame(df_final.groupby(['Operation'],as_index=False).size())
Division_count = pd.DataFrame(df_final.groupby(['Division'],as_index=False).size())
Designation_count = pd.DataFrame(df_final.groupby(['Designation'],as_index=False).size())

In [43]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)
df4=pd.DataFrame(groupby_operation)
df5=pd.DataFrame(groupby_division)
df6=pd.DataFrame(groupby_operation_sum)
df7=pd.DataFrame(groupby_division_sum)
df8=pd.DataFrame(groupby_designation)
df9=pd.DataFrame(groupby_designation_sum)

In [45]:
df4=df4.drop(['E.Code','Full Name','Designation','Division','Department','Date'], axis=1)
df5=df5.drop(['E.Code','Full Name','Designation','Operation','Department','Date'], axis=1)
df8=df8.drop(['E.Code','Full Name','Operation','Division','Department','Date'], axis=1)
df6=df6.drop(['E.Code'], axis=1) 
df7=df7.drop(['E.Code'], axis=1)
df9=df9.drop(['E.Code'], axis=1)

In [46]:
df4.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df5.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df8.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df6.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df7.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [47]:
l4=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l4.append(ot_time)

In [48]:
llp=[]
for i in df6.index:
      frac, whole = math.modf(df6['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      llp.append(ot_time)

In [49]:
lllp=[]
for i in df7.index:
      frac, whole = math.modf(df7['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      lllp.append(ot_time)

In [50]:
llllp=[]
for i in df9.index:
      frac, whole = math.modf(df9['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      llllp.append(ot_time)

In [51]:
df2['Total OT']=l4
df6['Total OT']=llp
df7['Total OT']=lllp
df9['Total OT']=llllp

In [52]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [53]:
df2=df2.drop(['OT'], axis=1)
df6=df6.drop(['OT'], axis=1)
df7=df7.drop(['OT'], axis=1)
df9=df9.drop(['OT'], axis=1)

In [54]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [55]:
lp=[]
for i in range(len(df6)):
  lp.append(df6.iloc[i]['Total OT']/df4.iloc[i]['Total OT_Days'])
lp2=[]
for i in range(len(df7)):
  lp2.append(df7.iloc[i]['Total OT']/df5.iloc[i]['Total OT_Days'])
lp3=[]
for i in range(len(df9)):
  lp3.append(df9.iloc[i]['Total OT']/df8.iloc[i]['Total OT_Days'])

In [57]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
# result = pd.concat([df3,df2], axis=1)
result_2 =  pd.concat([df4[['Operation','Total OT_Days']],df6['Total OT'],Operation_count['size']], axis=1)
result_3 =  pd.concat([df5,df7['Total OT'],Division_count['size']], axis=1)
result_4 =  pd.concat([df8,df9['Total OT'],Designation_count['size']], axis=1)

In [58]:
result_2.rename(columns={'size': 'Count'}, inplace=True)
result_3.rename(columns={'size': 'Count'}, inplace=True)
result_4.rename(columns={'size': 'Count'}, inplace=True)

In [59]:
ll2=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df3)):
  # Function call
  res = binarySearch(arr, 0, len(arr)-1, df3.iloc[i]['E.Code'])

  if res != -1:
      ll2.append([dataset2.iloc[res]['Full Name'], dataset2.iloc[res]['Designation'],dataset2.iloc[res]['Operation'],dataset2.iloc[res]['Division'],dataset2.iloc[res]['Department']])
  else:
      ll2.append(['NaN','NaN','NaN','NaN'])


In [60]:
# Converting 2d list to dataframe to apply CRUD opertations
dfff = pd.DataFrame(ll2, columns=['Full Name','Designation','Operation','Division','Department'])

In [61]:
result = pd.concat([df3['E.Code'], dfff, df3['Total OT_Days'], df2['Total OT']], axis=1)

In [62]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

,E.Code,Full Name,Designation,Operation,Division,Department,Total OT_Days,Total OT
0,32,Sanjeev Juyal,Chief Manager,General and Corporate Affairs,Industrial Relations,Employee Benefit,2,0.33
1,48,Sanjul Saxena,General Manager,Customer Service,CS Technical,NaN,1,0.13
2,224,Ravinder Kumar Sejwal,General Manager,Finance & Accounts,Accounts,NaN,1,1.11
3,314,Rajnish Charandas Mittal,General Manager,Customer Service,CS Field Service,NaN,2,1.03
4,332,M S Srivathsa,General Manager,Customer Service,Regional Business South CS,NaN,4,1.12
...,...,...,...,...,...,...,...,...
350,30009223,Raghubir .,Apprentice,Manesar Plant,Production Body Manesar,Paint,4,12.57
351,30009285,Pawan Kumar,Apprentice,Manesar Plant,Quality Control Manesar,Vehicle Quality,3,10.12
352,30009484,Sahil .,Apprentice,Manesar Plant,Production Body Manesar,Paint,1,3.45
353,30009499,Ashish .,Apprentice,Manesar Plant,Production Body Manesar,Press & Weld,2,7.08


In [63]:
result_hr_min=result.copy(deep=True)
l_time2=[]
for i in result_hr_min.index:
      minutes, hours = math.modf(result_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time2.append(ot_time)
result_hr_min.drop(['Total OT'],axis=1)
result_hr_min['Total OT']=l_time2

In [65]:
l_diff=[]
if date_diff==88 or 89 or 90 or 91 or 92:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>50:
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])
elif date_diff==28 or 29 or 30 or 31:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>16:
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])
else:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>((16/30)*date_diff):
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])


In [66]:
l_diff = pd.DataFrame(l_diff, columns=['E.Code','Full Name','Designation','Operation','Division','Department','Total OT_Days','Total OT'])

In [67]:
l_diff

,E.Code,Full Name,Designation,Operation,Division,Department,Total OT_Days,Total OT


In [68]:
l_diff_hr_min=l_diff.copy(deep=True)
l_time3=[]
for i in l_diff_hr_min.index:
      minutes, hours = math.modf(l_diff_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time3.append(ot_time)
l_diff_hr_min.drop(['Total OT'],axis=1)
l_diff_hr_min['Total OT']=l_time3

In [71]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
cont2=result.sort_values(by=['Total OT_Days'], ascending=False)

In [72]:
cont2_hr_min=cont2.copy(deep=True)
l_time4=[]
for i in cont2_hr_min.index:
      minutes, hours = math.modf(cont2_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time4.append(ot_time)
cont2_hr_min.drop(['Total OT'],axis=1)
cont2_hr_min['Total OT']=l_time4

In [73]:
cont2_consecutive=[]
i=0
while cont2.iloc[i]['Total OT_Days']>=10:
  cont2_consecutive.append(cont2.iloc[i]['E.Code'])
  i+=1

In [74]:
cont2_consecutive
l_con=[]
for x in cont2_consecutive:
  for j in range(len(df_final)):
    if x == df_final.iloc[j]['E.Code']:
      l_con.append([x, df_final.iloc[j]['Date']])


In [75]:
l_10con=[]
for j in cont2_consecutive:
  p,c=0,1
  for i in range(len(l_con)):
    p=i+1
    if p <= len(l_con)-1:
      if j==l_con[i][0] and j==l_con[p][0]:
          if l_con[i][1].day-l_con[p][1].day==-1:
            c+=1
          elif l_con[i][1].weekday()==4:
            if l_con[i][1].day-l_con[p][1].day==-3:
              c+=1
  if c>=10:
    l_10con.append(j)

In [76]:
l_10_final=[]
# result_10day=result
# result_10day=result_10day.drop(['Total OT_Days','Total OT'],axis=1)
j=0
while j<len(l_10con):
  for i in range(len(result)):
    if result.iloc[i]['E.Code']== l_10con[j]:
      l_10_final.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department']])
      break;
  j+=1

dfff_10 = pd.DataFrame(l_10_final, columns=['E.Code','Full Name','Designation','Operation','Division','Department'])


In [77]:
dfff_10

,E.Code,Full Name,Designation,Operation,Division,Department


In [78]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
cont3=result.sort_values(by=['Total OT'], ascending=False)

In [79]:
cont3_hr_min=cont3.copy(deep=True)
l_time5=[]
for i in cont3_hr_min.index:
      minutes, hours = math.modf(cont3_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time5.append(ot_time)
cont3_hr_min.drop(['Total OT'],axis=1)
cont3_hr_min['Total OT']=l_time5

In [80]:
cont4=result_2.sort_values(by=['Total OT_Days'], ascending=False)

In [81]:
cont4_hr_min=cont4.copy(deep=True)
l_time6=[]
for i in cont4_hr_min.index:
      minutes, hours = math.modf(cont4_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time6.append(ot_time)
cont4_hr_min.drop(['Total OT'],axis=1)
cont4_hr_min['Total OT']=l_time6

In [82]:
cont5=result_2.sort_values(by=['Total OT'], ascending=False)

In [83]:
cont5_hr_min=cont5.copy(deep=True)
l_time7=[]
for i in cont5_hr_min.index:
      minutes, hours = math.modf(cont5_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time7.append(ot_time)
cont5_hr_min.drop(['Total OT'],axis=1)
cont5_hr_min['Total OT']=l_time7

In [84]:
cont6=result_3.sort_values(by=['Total OT_Days'], ascending=False)

In [85]:
cont6_hr_min=cont6.copy(deep=True)
l_time8=[]
for i in cont6_hr_min.index:
      minutes, hours = math.modf(cont6_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time8.append(ot_time)
cont6_hr_min.drop(['Total OT'],axis=1)
cont6_hr_min['Total OT']=l_time8

In [86]:
cont7=result_3.sort_values(by=['Total OT'], ascending=False)

In [87]:
cont7_hr_min=cont7.copy(deep=True)
l_time9=[]
for i in cont7_hr_min.index:
      minutes, hours = math.modf(cont7_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time9.append(ot_time)
cont7_hr_min.drop(['Total OT'],axis=1)
cont7_hr_min['Total OT']=l_time9

In [88]:
cont8=result_4.sort_values(by=['Total OT_Days'], ascending=False)

In [89]:
cont8_hr_min=cont8.copy(deep=True)
l_time10=[]
for i in cont8_hr_min.index:
      minutes, hours = math.modf(cont8_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time10.append(ot_time)
cont8_hr_min.drop(['Total OT'],axis=1)
cont8_hr_min['Total OT']=l_time10

In [90]:
cont9=result_4.sort_values(by=['Total OT'], ascending=False)

In [91]:
cont9_hr_min=cont9.copy(deep=True)
l_time11=[]
for i in cont9_hr_min.index:
      minutes, hours = math.modf(cont9_hr_min['Total OT'][i])
      minutes*=100
      ot_time = "{:.0f} Hr {:.0f} Min".format(hours, minutes)
      l_time11.append(ot_time)
cont9_hr_min.drop(['Total OT'],axis=1)
cont9_hr_min['Total OT']=l_time11

In [92]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
from openpyxl import Workbook
import xlsxwriter
import os

workbook = xlsxwriter.Workbook(r'OT_Analysis.xlsx')
with pd.ExcelWriter(r'OT_Analysis.xlsx') as writer:
#writer = r"C:\Users\Kartikey\Desktop\shorttt.xlsx"
# write dataframe to excel
    df_final_hr_min.to_excel(writer,sheet_name='OT')
    result_hr_min.to_excel(writer,sheet_name='Total_OT')
    cont2_hr_min.to_excel(writer,sheet_name='Highest_Total_OT_Days')
    cont3_hr_min.to_excel(writer,sheet_name='Highest_Total_OT')
    l_diff_hr_min.to_excel(writer,sheet_name='Exceeding OT Limit')
    dfff_10.to_excel(writer,sheet_name='Exceeding 10 cons. OT days')
    l_double_shift2.to_excel(writer,sheet_name='Double shift')
    cont4_hr_min.to_excel(writer,sheet_name='Operation_Highest_Total_OT_Days')
    cont5_hr_min.to_excel(writer,sheet_name='Operation_Highest_Total_OT')
    cont6_hr_min.to_excel(writer,sheet_name='Division_Highest_Total_OT_Days')
    cont7_hr_min.to_excel(writer,sheet_name='Division_Highest_Total_OT')
    cont8_hr_min.to_excel(writer,sheet_name='Desig_Highest_Total_OT_Days')
    cont9_hr_min.to_excel(writer,sheet_name='Desig_Highest_Total_OT')